# Network Pruning

Network pruning is a commonly-used technique to speed up your model during inference. We will talk about this topic in this tutorial.

## Basic concept
As we all know, the majority of the runtime is attributed to the generic matrix multiply (a.k.a. GEMM) operations. So naturally, the problem comes out that whether we can speed up the operation by reducing the number of the elements in the matrices. By setting the weights, biases and the corresponding input and output items to 0, we can then just skip those calculations.

众所周知，大多数运行耗时都集中在通用矩阵乘法（又称GEMM）操作。所以，我们是否可以通过减少矩阵中元素的数量来加速运算。通过将权重、偏差和相应的输入和输出项设置为0，我们可以跳过这些计算。

There are generally two kinds of pruning, structured pruning and unstructured pruning. For structured pruning, the weight connections are removed in groups. e.g. The entire channel is deleted. It has the effect of changing the input and output shapes of layers and the weight matrices. Because of this, nearly every system can benefit from it. Unstructured pruning, on the other hand, removes individual weight connections from a network by setting them to 0. So, it is highly dependent on the inference backends. 

通常有两种修剪，结构化修剪和非结构化修剪。对于结构化剪枝，分组删除权重连接。e、 g.删除整个通道。它可以改变层的输入和输出形状以及权重矩阵。因此，几乎每个系统都可以从中受益。另一方面，非结构化剪枝通过将单个权重连接设置为0，从网络中删除它们。因此，它高度依赖于推理后端。

Currently, only structured pruning is supported in TinyNeuralNetwork.

### How structured pruning is implemented in DNN frameworks?
```py
model = Net(pretrained=True)
sparsity = 0.5

masks = {None: None}

def register_masks(layer):
    parent_layer = get_parent(layer)
    input_mask = masks[parent_layer]
    if is_passthrough_layer(layer):
        output_mask = input_mask
    else:
        output_mask = get_mask(layer, sparsity)
        register_mask(layer, input_mask, output_mask)
    masks[layer] = output_mask

model.apply(register_masks)
model.fit(train_data)

def apply_masks(layer):
    parent_layer = get_parent(layer)
    input_mask = masks[parent_layer]
    output_mask = masks[layer]
    apply_mask(layer, input_mask, output_mask)

model.apply(apply_masks)
```

### Network Pruning in TinyNerualNetwork
The problem in the previous code example is that only one parent layer is expected. But in some recent DNN models, there are a few complicated operations like `cat`, `add` and `split`. We need to resolve the dependencies of those operations as well.

前面代码示例中的问题是只期望有一个父层。但在最近的一些DNN模型中，有一些复杂的操作，如“cat”、“add”和“split”。我们还需要解决这些操作的依赖关系。

To solve the aforementioned problem, first we go through some basic definitions. When the input shape and output shape of a node are not related during pruning, it is called a node with isolation. For example, the `conv`, `linear` and `lstm` nodes are nodes with isolation. We want to find out a group of nodes, which is called a subgraph, that starts with and ends with nodes with isolation and doesn't contain a subgraph in it. We use the nodes with isolation for finding out the candidate subgraphs in the model. 

要解决上述问题，首先我们要了解一些基本的定义。当剪枝过程中某个节点的输入形状和输出形状不相关时，称为“隔离节点”。例如，' conv '、' linear '和' lstm '节点是具有隔离的节点。我们希望找到一组节点，即所谓的子图，它以具有隔离的节点开始并以节点结束，并且其中不包含子图。我们使用具有隔离的节点来找出模型中的候选子图。


```py
def find_subgraph(layer, input_modify, output_modify, nodes):
    if layer in nodes:
        return None

    nodes.append(layer)

    if is_layer_with_isolation(layer):
        if input_modify:
            for prev_layer in get_parent(layer):
                return find_subgraph(prev_layer, False, True, nodes)
        if output_modify:
            for next_layer in get_child(layer):
                return find_subgraph(next_layer, True, False, nodes)
    else:
        for prev_layer in get_parent(layer):
            return find_subgraph(prev_layer, input_modify, output_modify, nodes)
        for next_layer in get_child(layer):
            return find_subgraph(next_layer, input_modify, output_modify, nodes)

candidate_subgraphs = []

def construct_candidate_subgraphs(layer):
    if is_layer_with_isolation(layer):
        nodes = []
        find_subgraph(layer, True, False, nodes)
        candidate_subgraphs.append(nodes)

        nodes = []
        find_subgraph(layer, False, True, nodes)
        candidate_subgraphs.append(nodes)

model.apply(construct_subgraphs)
```

With all candidate subgraphs, the next step we do is to remove the duplicated and invalid ones in them. Due to space limitations, we will not cover this section in detail. When we get the final subgraphs, the first node in it is called the center node. During configuration, we use the name of the center node to represent the subgraph it constructs. Some properties can be set at the subgraph level by the user, like sparsity.

对于所有的候选子图，下一步我们要做的是删除其中的重复和无效的子图。由于篇幅的限制，我们将不详细介绍这一节。当我们得到最终的子图时，其中的第一个节点称为中心节点。在配置期间，我们使用中心节点的名称来表示它构造的子图。用户可以在子图级别设置一些属性，比如稀疏性。

Although we have the subgraphs, the mapping of channels between nodes is still unknown. So we need to resolve channel dependency. Similarly, we pass the channel information recursively so as to get the correct mapping at each node. It may be a bit more complicated since each node has its own logic for sharing channel mapping. Operations like `add` require shared mapping in all the input and output tensors, while `cat` allows the inputs to have independent mappings, however the output mapping and the combined input mapping is shared. As this is too detailed, we will not expand on it.

虽然我们有子图，但节点之间的通道映射仍然是未知的。因此我们需要解决通道依赖问题。类似地，我们递归地传递通道信息，以便在每个节点获得正确的映射。这可能有点复杂，因为每个节点都有自己的共享通道映射逻辑。像' add '这样的操作需要在所有输入和输出张量中共享映射，而' cat '允许输入具有独立映射，但是输出映射和组合输入映射是共享的。因为太详细了，我们就不展开了。

After resolving the channel dependency, we follow the ordinary pruning process, that is to register the masks of the weight and bias tensors. And then you may just finetune the model. When the training process is finished, then it is time to apply the masks, so that the model actually gets smaller. Alternatively, you may apply the masks just after registering them if the masks won't change during training. As a result, the training process will be significantly faster. That's all the story for pruning.

在解决了信道依赖问题后，我们遵循普通的剪枝过程，即注册权重、偏置张量的掩码，然后对模型进行微调。当训练过程完成时，就该应用掩码了，这样模型就会变得更小。或者，如果在培训期间掩码不变，可以在注册后立即应用掩码。因此，训练过程将大大加快。这就是剪枝的全部内容。

### Using the pruner in TinyNeuralNetwork
It is really simple to use the pruner in our framework. You can use the code below.


In [4]:
import sys
sys.path.append('../..')

import torch
import torchvision

from tinynn.prune.oneshot_pruner import OneShotChannelPruner

model = torchvision.models.mobilenet_v2(pretrained=True)
model.train()

dummy_input = torch.randn(1, 3, 224, 224)

pruner = OneShotChannelPruner(model, dummy_input, config={'sparsity': 0.25, 'metrics': 'l2_norm'})

st_flops = pruner.calc_flops()
pruner.prune()

ed_flops = pruner.calc_flops()
print(f"Pruning over, reduced FLOPS {100 * (st_flops - ed_flops) / st_flops:.2f}%  ({st_flops} -> {ed_flops})")

# You should start finetuning the model here

INFO (tinynn.graph.modifier) [CONV] features_0_0: output 32 -> 24
INFO (tinynn.graph.modifier) [BN] features_0_1: channel 32 -> 24
INFO (tinynn.graph.modifier) [DW_CONV] features_1_conv_0_0: input 32 -> 24
INFO (tinynn.graph.modifier) [BN] features_1_conv_0_1: channel 32 -> 24
INFO (tinynn.graph.modifier) [CONV] features_1_conv_1: input 32 -> 24
INFO (tinynn.graph.modifier) [CONV] features_1_conv_1: output 16 -> 12
INFO (tinynn.graph.modifier) [BN] features_1_conv_2: channel 16 -> 12
INFO (tinynn.graph.modifier) [CONV] features_2_conv_0_0: input 16 -> 12
INFO (tinynn.graph.modifier) [CONV] features_2_conv_0_0: output 96 -> 72
INFO (tinynn.graph.modifier) [BN] features_2_conv_0_1: channel 96 -> 72
INFO (tinynn.graph.modifier) [DW_CONV] features_2_conv_1_0: input 96 -> 72
INFO (tinynn.graph.modifier) [BN] features_2_conv_1_1: channel 96 -> 72
INFO (tinynn.graph.modifier) [CONV] features_2_conv_2: input 96 -> 72
INFO (tinynn.graph.modifier) [CONV] features_2_conv_2: output 24 -> 18
INFO (

Pruning over, reduced FLOPS 40.99%  (314130496 -> 185359152)
